In [ ]:
!pip install llama-index-llms-groq
!pip install llama-index
!pip install pdfplumber
!pip install scikit-learn

In [ ]:
from llama_index.llms.groq import Groq
import pdfplumber
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np


In [ ]:
# Create an API key at the Groq console (https://console.groq.com/keys)
# Then set it to the environment variable GROQ_API_KEY or pass it directly

api_key = "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"  # Replace with your actual API key
llm = Groq(model="llama3-70b-8192", api_key=api_key)


In [ ]:
def extract_text_with_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Example usage
pdf_path = "/content/agenda.pdf"  # Replace with the path to your PDF file
extracted_text = extract_text_with_pdfplumber(pdf_path)
print(extracted_text)


Agenda
Global IndiaAI Summit 2024 3-4 July 2024, New Delhi
Day 1 - Wednesday, July 3
Start Session Description
10:00 - 11:00
Opening Ceremony (Auditorium 1)
11:00 – 13:30
GPAI Session (@ Summit Room) : Executive Council Meeting
(By Invitation only)
11:30 - 13:00 Side Event 1 (@ The session is about learning and sharing knowledge on Large
Auditorium 1): Language Models (LLMs) and Large Multimodal Models (LMMs),
with the aim to understand the unique linguistic and cultural
IndiaAI: Large diversity inherent to India, examining how LLMs can effectively
Language Model address challenges associated with multilingualism. Furthermore,
the session will cover the ethical considerations and biases linked
to these models, promoting a discussion on responsible AI
practices, including fairness, inclusiveness, misinformation
mitigation, and intellectual property rights within diverse cultural
contexts. Lastly, the session will focus on the collaborative
opportunities among indigenous communities, aca

In [ ]:
# Define a list of questions and their expected answers (true labels)
questions_and_answers = [
    ("What is the name of the summit?", "Global IndiaAI Summit 2024"),
    ("When and where is the Global IndiaAI Summit 2024 taking place?", "3-4 July 2024, New Delhi"),
    ("What is the time and venue for the opening ceremony on Day 1?", "10:00 - 11:00 at Auditorium 1"),
    ("What is the GPAI Session about at 11:00 on Day 1?", "Executive Council Meeting (By Invitation only)"),
]

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
# Function to ask questions and get predictions from the model
def ask_questions(text, questions_and_answers):
    true_labels = []
    predicted_labels = []

    for question, true_answer in questions_and_answers:
        true_labels.append(true_answer)
        response = llm.complete(f" Give one word answers based on {text}, for the question: {question}")
        predicted_labels.append(response.text.strip())  # Accessing the text attribute directly

    return true_labels, predicted_labels

# Example usage
true_labels, predicted_labels = ask_questions(extracted_text, questions_and_answers)
print("True Labels:")
print(true_labels)
print("Predicted Labels:")
print(predicted_labels)


True Labels:
['Global IndiaAI Summit 2024 is happening in New Delhi on 3rd and 4th July.']
Predicted Labels:
['Here are the names of events mentioned in the agenda:\n\n1. Opening Ceremony\n2. GPAI Session: Executive Council Meeting\n3. Side Event 1: Large Language Models (LLMs) and Large Multimodal Models (LMMs)\n4. Side Event 2: GPAI Convening on Global Health and AI\n5. GPAI Session: Ministerial Council Meeting\n6. Side Event 3: Real World AI Solutions\n7. Side Event 4: India’s Infrastructure Readiness for AI\n8. Side Event 5: Ensuring Safety, Trust, and Governance in the AI Age\n9. Side Event 6: Collaborative AI on Global Partnership (CAIGP)\n10. Side Event 7: Empowering Talent through AI Education & Skilling\n11. Side Event 8: AI for Global Good: Empowering the Global South\n12. Side Event 9: From Seed to Scale - Empowering India’s Startup Ecosystem\n13. GPAI Session: Innovation Workshop Followup\n14. Side Event 10: Data Ecosystem\n15. Side Event 11: AI Competency Framework for Pub

In [ ]:
# Function to evaluate model using token-level comparisons
def evaluate_model(true_labels, predicted_labels):
    correct_count = 0
    total_count = len(true_labels)

    for true_label, predicted_label in zip(true_labels, predicted_labels):
        # Tokenize and convert to sets
        true_tokens = set(true_label.lower().split())
        predicted_tokens = set(predicted_label.lower().split())

        # Calculate intersection
        intersection = true_tokens.intersection(predicted_tokens)

        # If there is any intersection, count it as correct (lenient matching)
        if intersection:
            correct_count += 1

    precision = correct_count / total_count
    recall = correct_count / total_count
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

precision, recall, f1 = evaluate_model(true_labels, predicted_labels)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
# Print each question, true answer, and predicted answer
print("Comparison of True and Predicted Answers:")
for i, (true_label, predicted_label) in enumerate(zip(true_labels, predicted_labels)):
    print(f"Question {i + 1}:")
    print(f"True Answer: {true_label}")
    print(f"Predicted Answer: {predicted_label}")
    if true_label != predicted_label:
        print("--> Score impacted: Check precision and recall for this question.")
    print()


Comparison of True and Predicted Answers:
Question 1:
True Answer: Global IndiaAI Summit 2024 is happening in New Delhi on 3rd and 4th July. Agenda of the events are shared.
Predicted Answer: Here is a 30-word summary of the Global IndiaAI Summit 2024 agenda:

The 2-day summit features 12 side events, 3 GPAI sessions, and an opening ceremony, covering AI topics like language models, healthcare, infrastructure, governance, and more.
--> Score impacted: Check precision and recall for this question.



In [ ]:
# Function to calculate precision, recall, and F1 score for each answer
def calculate_scores(true_label, predicted_label):
    true_tokens = set(true_label.lower().split())
    predicted_tokens = set(predicted_label.lower().split())

    intersection = true_tokens.intersection(predicted_tokens)
    precision = len(intersection) / len(predicted_tokens) if predicted_tokens else 0
    recall = len(intersection) / len(true_tokens) if true_tokens else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Function to evaluate model using token-level comparisons and print details
def evaluate_and_print(true_labels, predicted_labels):
    total_precision, total_recall, total_f1 = 0, 0, 0
    total_count = len(true_labels)

    print("Comparison of True and Predicted Answers with Scores:")
    for i, (true_label, predicted_label) in enumerate(zip(true_labels, predicted_labels)):
        precision, recall, f1 = calculate_scores(true_label, predicted_label)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

        print(f"Question {i + 1}:")
        print(f"True Answer: {true_label}")
        print(f"Predicted Answer: {predicted_label}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print()

    avg_precision = total_precision / total_count
    avg_recall = total_recall / total_count
    avg_f1 = total_f1 / total_count

    print(f"Average Precision: {avg_precision:.2f}")
    print(f"Average Recall: {avg_recall:.2f}")
    print(f"Average F1 Score: {avg_f1:.2f}")

# Example usage
evaluate_and_print(true_labels, predicted_labels)


Comparison of True and Predicted Answers with Scores:
Question 1:
True Answer: Global IndiaAI Summit 2024 is happening in New Delhi on 3rd and 4th July.
Predicted Answer: Here is the summary of the Global IndiaAI Summit 2024:

**Day 1 - Wednesday, July 3**

1. Opening Ceremony
2. GPAI Session: Executive Council Meeting (By Invitation only)
3. Side Event 1: Large Language Models and Large Multimodal Models
4. Side Event 2: GPAI Convening on Global Health and AI
5. GPAI Session: Ministerial Council Meeting (By Invitation only)
6. Side Event 3: Real World AI Solutions
7. Side Event 4: India's Infrastructure Readiness for AI
8. Side Event 5: Ensuring Safety, Trust, and Governance in the AI Age
9. Side Event 6: Collaborative AI on Global Partnership (CAIGP)

**Day 2 - Thursday, July 4**

1. Side Event 7: Empowering Talent through AI Education & Skilling
2. Side Event 8: AI for Global Good: Empowering the Global South
3. Side Event 9: From Seed to Scale - Empowering India's Startup Ecosystem

In [ ]:
# Function to calculate ROUGE scores
def calculate_rouge(true_label, predicted_label):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(true_label, predicted_label)
    return scores

# Function to evaluate model using ROUGE scores
def evaluate_with_rouge(true_labels, predicted_labels):
    total_rouge1, total_rouge2, total_rougeL = 0, 0, 0
    count = len(true_labels)

    for true_label, predicted_label in zip(true_labels, predicted_labels):
        scores = calculate_rouge(true_label, predicted_label)
        total_rouge1 += scores['rouge1'].fmeasure
        total_rouge2 += scores['rouge2'].fmeasure
        total_rougeL += scores['rougeL'].fmeasure

    avg_rouge1 = total_rouge1 / count
    avg_rouge2 = total_rouge2 / count
    avg_rougeL = total_rougeL / count

    return avg_rouge1, avg_rouge2, avg_rougeL

# Example usage
avg_rouge1, avg_rouge2, avg_rougeL = evaluate_with_rouge(true_labels, predicted_labels)
print(f"ROUGE-1: {avg_rouge1:.2f}")
print(f"ROUGE-2: {avg_rouge2:.2f}")
print(f"ROUGE-L: {avg_rougeL:.2f}")


ROUGE-1: 0.09
ROUGE-2: 0.03
ROUGE-L: 0.07
